In [43]:
import pandas as pd
import requests
import sqlite3
from rdfpandas.graph import to_graph
from rdflib import Graph, Namespace
from dotenv import load_dotenv
import os

load_dotenv()

True

In [29]:
dataset_id = "d_17f5382f26140b1fdae0ba2ef6239d2f"
url = "https://data.gov.sg/api/action/datastore_search?resource_id="  + dataset_id
        
response = requests.get(url)
print(response.json())

{'help': 'https://data.gov.sg/api/3/action/help_show?name=datastore_search', 'success': True, 'result': {'resource_id': 'd_17f5382f26140b1fdae0ba2ef6239d2f', 'fields': [{'type': 'text', 'id': 'blk_no'}, {'type': 'text', 'id': 'street'}, {'type': 'text', 'id': 'max_floor_lvl'}, {'type': 'text', 'id': 'year_completed'}, {'type': 'text', 'id': 'residential'}, {'type': 'text', 'id': 'commercial'}, {'type': 'text', 'id': 'market_hawker'}, {'type': 'text', 'id': 'miscellaneous'}, {'type': 'text', 'id': 'multistorey_carpark'}, {'type': 'text', 'id': 'precinct_pavilion'}, {'type': 'text', 'id': 'bldg_contract_town'}, {'type': 'text', 'id': 'total_dwelling_units'}, {'type': 'text', 'id': '1room_sold'}, {'type': 'text', 'id': '2room_sold'}, {'type': 'text', 'id': '3room_sold'}, {'type': 'text', 'id': '4room_sold'}, {'type': 'text', 'id': '5room_sold'}, {'type': 'text', 'id': 'exec_sold'}, {'type': 'text', 'id': 'multigen_sold'}, {'type': 'text', 'id': 'studio_apartment_sold'}, {'type': 'text', '

In [30]:
df_hdb_info = pd.read_csv("data/HDBPropertyInformation.csv")
df_hdb_info.shape

(13267, 24)

In [31]:
# Create SQLite database from CSV
conn = sqlite3.connect('hdb_db.db')
df_hdb_info.to_sql('hdb_db', conn, if_exists='replace', index=False)
conn.commit()
print("Database created successfully!")
print(f"Total records in database: {len(df_hdb_info)}")
print(f"Columns: {list(df_hdb_info.columns)}")

Database created successfully!
Total records in database: 13267
Columns: ['blk_no', 'street', 'max_floor_lvl', 'year_completed', 'residential', 'commercial', 'market_hawker', 'miscellaneous', 'multistorey_carpark', 'precinct_pavilion', 'bldg_contract_town', 'total_dwelling_units', '1room_sold', '2room_sold', '3room_sold', '4room_sold', '5room_sold', 'exec_sold', 'multigen_sold', 'studio_apartment_sold', '1room_rental', '2room_rental', '3room_rental', 'other_room_rental']


In [32]:
# Check what tables actually exist in your database
res = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(res)

     name
0  hdb_db


In [33]:
# Load the DataFrame into the SQLite database
df_hdb_info.to_sql('hdb_db', conn, index=False, if_exists='replace')

# Execute a SQL query
query = "SELECT * FROM hdb_db WHERE street = 'CHAI CHEE RD';"
result = pd.read_sql_query(query, conn)

# Display the result
print(result)

   blk_no        street  max_floor_lvl  year_completed residential commercial  \
0       1  CHAI CHEE RD             15            1982           Y          N   
1      10  CHAI CHEE RD             15            1999           Y          Y   
2     10A  CHAI CHEE RD              1            1999           N          N   
3      11  CHAI CHEE RD             15            1999           Y          Y   
4      19  CHAI CHEE RD             10            1972           Y          N   
5       2  CHAI CHEE RD             12            1982           Y          N   
6      20  CHAI CHEE RD             12            1972           Y          Y   
7      21  CHAI CHEE RD             12            1972           Y          Y   
8     21A  CHAI CHEE RD              5            2001           N          N   
9      22  CHAI CHEE RD             12            1972           Y          N   
10     23  CHAI CHEE RD             12            1972           Y          N   
11    23A  CHAI CHEE RD     

In [38]:
from rdflib import Graph, Literal, RDF, URIRef, Namespace
from rdflib.namespace import RDFS, XSD
from urllib.parse import quote

# 1. Setup Namespace with valid URI format
g = Graph()
HDB = Namespace("http://hdb_property_inforomation_data.gov/")
HDBPROP = Namespace("http://hdb_property_inforomation_data.gov/property/")
g.bind("hdb", HDB)
g.bind("hdbprop", HDBPROP)

# 2. Create Class definition for HDB Property
g.add((HDB.HDBProperty, RDF.type, RDFS.Class))
g.add((HDB.HDBProperty, RDFS.label, Literal("HDB Property")))
g.add((HDB.HDBProperty, RDFS.comment, Literal("Represents an HDB (Housing and Development Board) property in Singapore")))

# 3. Get columns from the DataFrame
columns = df_hdb_info.columns.tolist()

# 4. Generate Property Definitions
for col in columns:
    # Create safe URI by encoding special characters
    safe_col = quote(col, safe='')
    prop_uri = HDBPROP[safe_col]
    
    g.add((prop_uri, RDF.type, RDF.Property))
    g.add((prop_uri, RDFS.label, Literal(col)))
    g.add((prop_uri, HDB.sqlColumn, Literal(col)))
    
    # Determine data type from DataFrame
    dtype = str(df_hdb_info[col].dtype)
    if 'int' in dtype:
        g.add((prop_uri, RDFS.range, XSD.integer))
    elif 'float' in dtype:
        g.add((prop_uri, RDFS.range, XSD.decimal))
    else:
        g.add((prop_uri, RDFS.range, XSD.string))
    
    # Add domain
    g.add((prop_uri, RDFS.domain, HDB.HDBProperty))
    
    # Logic: If it starts with a number, it needs quotes in SQL
    if col[0].isdigit():
        g.add((prop_uri, HDB.requiresQuotes, Literal(True, datatype=XSD.boolean)))
    else:
        g.add((prop_uri, HDB.requiresQuotes, Literal(False, datatype=XSD.boolean)))

# 5. Save to Turtle file
g.serialize(destination='hdb_schema.ttl', format='turtle')
print(f"hdb_schema.ttl created successfully!")
print(f"Total properties mapped: {len(columns)}")
print(f"Total triples: {len(g)}")

hdb_schema.ttl created successfully!
Total properties mapped: 24
Total triples: 147


In [49]:
import google.genai as genai

In [52]:
client = genai.Client(api_key=os.getenv("GENAI_API_KEY"))
gemini_3_flash = "gemini-3-flash-preview"


In [53]:
response = client.models.generate_content(
    model=gemini_3_flash, contents="Explain how AI works in a few words"
)
print(response.text)

**Finding patterns in data to make predictions.**


In [61]:
# Step 1: Load the Turtle ontology
with open('hdb_schema.ttl', 'r') as f:
    turtle_ontology = f.read()

# Step 2: Define the semantic-grounded query function
def query_hdb_with_semantics(user_query):
    """
    Process user query using LLM with semantic grounding from Turtle ontology
    Returns: extracted entities, generated SQL, and query results
    """
    
    # Create the grounding prompt with ontology
    grounding_prompt = f"""You are a SQL expert helping users query an HDB (Housing and Development Board) property database.

Here is the database schema ontology:
{turtle_ontology}

The database table is called 'hdb_db' with columns from the ontology above.

User Query: {user_query}

Tasks:
1. Extract the relevant entities/properties from the query
2. Map them to the appropriate columns in the database
3. Generate a valid SQLite SQL query

Respond in this exact format:
ENTITIES: [list the extracted entities]
MAPPED_COLUMNS: [list the corresponding database columns]
SQL_QUERY: [the actual SQL query]
EXPLANATION: [brief explanation of what the query does]"""

    # Call LLM to process query with semantic grounding
    response = client.models.generate_content(
        model=gemini_3_flash,
        contents=grounding_prompt
    )
    
    llm_response = response.text
    print("=" * 60)
    print("LLM SEMANTIC ANALYSIS:")
    print("=" * 60)
    print(llm_response)
    print("=" * 60)
    
    # Extract SQL query from LLM response
    sql_query = None
    for line in llm_response.split('\n'):
        if 'SQL_QUERY:' in line:
            # Extract text after SQL_QUERY:
            sql_query = line.split('SQL_QUERY:')[1].strip().rstrip(';')
            break
    
    if not sql_query:
        print("No valid SQL query generated")
        return None
    
    # Step 3: Execute the SQL query
    print(f"\nExecuting SQL: {sql_query}\n")
    try:
        result_df = pd.read_sql_query(sql_query, conn)
        print(f"Results ({len(result_df)} rows):")
        print(result_df.to_string())
        return {
            'user_query': user_query,
            'llm_analysis': llm_response,
            'sql_query': sql_query,
            'results': result_df
        }
    except Exception as e:
        print(f"Error executing query: {e}")
        return None

# Step 3: Test with a sample query
sample_query = "Show me all HDB blocks on BEACH RD and their number of 3-room units sold"
result = query_hdb_with_semantics(sample_query)

LLM SEMANTIC ANALYSIS:
ENTITIES: blocks, BEACH RD, 3-room units sold
MAPPED_COLUMNS: blk_no, street, "3room_sold"
SQL_QUERY: SELECT blk_no, "3room_sold" FROM hdb_db WHERE street = 'BEACH RD';
EXPLANATION: This query selects the block numbers and the number of 3-room units sold for all HDB properties where the street name matches 'BEACH RD'. Note that "3room_sold" is enclosed in double quotes as specified by the schema requirements for columns starting with numbers.

Executing SQL: SELECT blk_no, "3room_sold" FROM hdb_db WHERE street = 'BEACH RD'

Results (10 rows):
  blk_no  3room_sold
0      1         138
1     14           0
2     15           0
3     17           0
4      2         134
5      3         134
6      4           0
7     4A           0
8      5           0
9      6           1


In [57]:
print(result)

None
